In [ ]:
import pandas as pd
from google.cloud import bigquery
from google.auth import default

In [ ]:
credentials, _ = default()
projectName = "daywork-215507"
hospitalId = 41
client = bigquery.Client(project=projectName, credentials=credentials)

In [ ]:
query = f"""
SELECT * FROM `{projectName}.hospital_prediction.fingerprints` LIMIT 10
"""

In [ ]:
query = f"""
    SELECT
        fp.id,
        fp.hospital_name,
        fp.hospital_zone_name,
        ARRAY_AGG(STRUCT(
            r.ssid,
            r.bssid,
            r.level,
            r.frequency
        )) AS radiation_records
    FROM
        `{projectName}.hospital_prediction.fingerprints` AS fp
    LEFT JOIN
        `{projectName}.hospital_prediction.wifi_radiations` AS r
    ON
        fp.id = r.fingerprint_id
    WHERE fp.is_active = true and fp.hospital_id = {hospitalId}
    GROUP BY
        fp.id, fp.hospital_zone_name, fp.hospital_name
"""

In [ ]:
query_job = client.query(query)
results = query_job.result()
# reses = [dict(res.items()) for res in results]

In [ ]:
# reses[0]

In [ ]:
fingerprints_with_radiation_admin = []
fingerprints_with_radiation_data = []
hospital_name = None

for row in results:
    if hospital_name is None:
        hospital_name = row.hospital_name

    if "crowd" not in row.hospital_zone_name.lower():
        fingerprints_with_radiation_admin.append(
            {
                "id": row.id,
                "point": row.hospital_zone_name,
                "dataDictAll": [
                    {
                        "ssid": rad["ssid"],
                        "bssid": rad["bssid"],
                        "level": rad["level"],
                        "frequency": rad["frequency"],
                    }
                    for rad in row.radiation_records
                ],
            }
        )
    else:
        fingerprints_with_radiation_data.append(
            {
                "id": row.id,
                "point": row.hospital_zone_name,
                "dataDictAll": [
                    {
                        "ssid": rad["ssid"],
                        "bssid": rad["bssid"],
                        "level": rad["level"],
                        "frequency": rad["frequency"],
                    }
                    for rad in row.radiation_records
                ],
            }
        )

In [ ]:
admin_json = fingerprints_with_radiation_admin

In [ ]:
df = pd.DataFrame.from_dict(admin_json)

In [ ]:
df["point"].value_counts()

In [ ]:
if hospitalId == 15:
    dfSup = df
    dfUnsup = pd.DataFrame(columns=df.columns) # Empty
elif hospitalId == 41:
    filtUnsup = df["point"] == ""
    dfSup = df[~filtUnsup]
    dfUnsup = df[filtUnsup]

In [ ]:
dfSup.to_json(f"./HID_{hospitalId}_sup.json", orient="records")
dfUnsup.to_json(f"./HID_{hospitalId}_unsup.json", orient="records")